# Detecting audio issues in a condition monitoring dataset (audio)
This notebook aims at detecting issues in a **condition monitoring** dataset using **audio data**. As a basis it uses the DCASE challenge dataset where the goal is to detect if a machine is in a defect state or not.

In order to run the example install the **dependencies** as follows:

In [1]:
!pip install datasets


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## Step 1: Download the dataset

In [1]:
# Import datasets to download the data
import datasets

In [2]:
# Download the dataset and convert to pandas dataframe
dataset = datasets.load_dataset(
        "renumics/dcase23-task2-enriched", "dev", split="all", streaming=False
    )
df = dataset.to_pandas()

Found cached dataset dcase23-task2-enriched (/home/daniel/.cache/huggingface/datasets/renumics___dcase23-task2-enriched/dev/3.0.0/c945b4ebc3162e1b75113e4813bc56f4eceed94d7e693d03c39f898c32d751bb)


In [3]:
# Sample the dataset randomly to make the example run faster
df = df.sample(1000)

# Step 2: Detect problematic data slices based on audio data

In [4]:
# The imports
from sklearn.metrics import accuracy_score
from renumics.spotlight import Audio
from sliceguard import SliceGuard

In [5]:
# Run slice detection based on general purpose audio embeddings (pretrained model trained on Audioset)
sg = SliceGuard()
issue_df = sg.find_issues(
    df,
    ["path"],
    "label",
    "dev_train_lof_anomaly",
    accuracy_score,
    metric_mode="max",
    min_support=5,
    min_drop=0.2,
)

Feature path was inferred as referring to raw data. If this is not the case, please specify in feature_types!
Using default model for computing embeddings for feature path.


Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.layernorm.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.layernorm.weight']
- This IS expected if you are initializing ASTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ASTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████| 1000/1000 [01:04<00:00, 15.51it/s]


The overall metric value is 0.889
Using 5 as minimum support for determining problematic clusters.
Using 0.2 as minimum drop for determining problematic clusters.
Identified 6 problematic slices.


In [6]:
# Report the results in Renumics Spotlight
sg.report(spotlight_dtype={"path": Audio})